In [1]:
from MT5 import *
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LinearRegression
import time
import pickle
from joblib import dump, load
import os

path = "" # Ex: C:/Desktop/Python_for_finance_and_algorithmic_trading/ChapterN/Models


def create_model_weights(symbol):
    """ Weights for Linear regression on the percentage change"""
    # Import the data
    data = MT5.get_data(symbol, 3500)[["close"]].pct_change(1)
    
    # Create new variable
    data["close t-1"] = data[["close"]].shift(1)
    
    # Split the data
    data = data.dropna()
    split = int(0.80*len(data))
    
    # Train set creation
    X_train = data[["close t-1"]].iloc[:split]
    y_train = data[["close"]].iloc[:split]

    # Test set creation
    X_test = data[["close t-1"]].iloc[split:]
    
    # Create the model
    alg = LinearRegression()

    # Fit the model
    alg.fit(X_train, y_train)
    
    # Save the model
    alg_var = pickle.dumps(alg)
    alg_pickel = pickle.loads(alg_var)

    dump(alg_pickel ,os.path.join(path,f"Models/{symbol}.joblib"))
    
    
    

def lin_reg_sig(symbol):
    """ Function for predict the value of tommorow using ARIMA model"""
    
    # Create the weights if there is not in the folder
    try:
        alg = load(os.path.join(path,f"Models/{symbol}.joblib"))
    except:
        create_model_weights(symbol)
        alg = load(os.path.join(path,f"Models/{symbol}.joblib"))
    
    # Take the lastest percentage of change 
    data = MT5.get_data(symbol, 3500)[["close"]].pct_change(1)
    X = data["close"].iloc[-1].reshape(-1,1)
    
    
    # Find the signal
    prediction = alg.predict(X)
    buy = prediction[0][0] > 0
    sell = not buy
    
    
    return buy, sell



# True = Live Trading and False = Screener
live = True

if live:
    current_account_info = mt5.account_info()
    print("------------------------------------------------------------------")
    print("Date: ", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    print(f"Balance: {current_account_info.balance} USD, \t"
          f"Equity: {current_account_info.equity} USD, \t"
          f"Profit: {current_account_info.profit} USD")
    print("------------------------------------------------------------------")




info_order = {
    "Euro vs USdollar": ["EURUSD", 0.01]
}


start = datetime.now().strftime("%H:%M:%S")
while True:
    # Verfication for launch
    if datetime.now().weekday() not in (5,6):
        is_time = datetime.now().strftime("%H:%M:%S") == start #"23:59:59"
    else:
        is_time = False

    
    # Launch the algorithm
    if is_time:

        # Open the trades
        for asset in info_order.keys():

            # Initialize the inputs
            symbol = info_order[asset][0]
            lot = info_order[asset][1]

            # Create the signals
            buy, sell = lin_reg_sig(symbol)

             # Run the algorithm
            if live:
                MT5.run(symbol, buy, sell,lot)

            else:
                print(f"Symbol: {symbol}\t"
                     f"Buy: {buy}\t"
                     f"Sell: {sell}")
    time.sleep(1)

------------------------------------------------------------------
Date:  2021-09-24 07:18:24
Balance: 982.03 USD, 	Equity: 982.03 USD, 	Profit: 0.0 USD
------------------------------------------------------------------
------------------------------------------------------------------
Date:  2021-09-24 07:18:24
SYMBOL: EURUSD
BUY: False 	  SHORT: True
POSITION: None 	 ID: None
OPEN SHORT TRADE: Request executed
------------------------------------------------------------------


KeyboardInterrupt: 